# Optimization Problem Time Series

This notebook optimizes an electrci mix based on : 
- an electricity demand time serie
- a PV production time serie
- a wind production time serie

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pulp import *
import sys
from utilities import import_excel
import plotly.graph_objects as go
import plotly.express as px
import pickle

import plotly.io as pio
pio.renderers.default='notebook'

Enter country name for file naming: 

In [2]:
country_name = 'France'
year = 2021

# Directory path
path_input_data = '../input_time_series/'

In [3]:
colors_dict = {
    'Wind': 'steelblue',        
    'PV': 'gold',
    'Discharge': 'orangered',    
    'SOC': 'darkgreen',           
    'Charge': 'purple',
    'Consumption': 'green',          
    'Dispatchable': 'crimson',       
    'Curtailment': 'cyan'    
}

### Load Time Series

First we load the time series that will be used in the problem. We use the  ```import_excel``` function used in the wavelet decomposition.

#### Demand

In [4]:
# Demand time serie

dpd = 24 # data per day in the time serie
dpy = 365 # data per year :  cut the leap years to 365 years

ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

# !!! CHANGE FILENAME HERE !!!
file_name = 'France/France_load_entsoe_2021.xlsx'

Load_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = 'mean') # interpolate data from dpd to ndpd numper of points per day

mean_load = pd.read_excel(path_input_data+file_name).mean().iloc[0]

#### Wind production

In [6]:
# Wind time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

# !!! CHANGE FILENAME HERE !!!
file_name = 'France/wind_onshore_generation_France_2019_opsd.xlsx'

Wind_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = 'max') # interpolate data from dpd to ndpd numper of points per day

mean_wind = pd.read_excel(path_input_data+file_name).mean().iloc[0]

In [7]:
pd.read_excel(path_input_data+file_name)

,FR_wind_onshore_generation_actual
0,1637.0
1,1567.0
2,1556.0
3,1595.0
4,1719.0
...,...
8755,2695.0
8756,2831.0
8757,3031.0
8758,3334.0


#### PV production

In [8]:
# PV time serie

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years


# We interpolate so that we have hourly data
ndpd = 24 # new data per day (for the interpolation)
signal_length = ndpd * dpy


# !!! CHANGE FILENAME HERE !!!
file_name = 'France/solar_generation_France_2019_opsd.xlsx'

PV_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = 'max') # interpolate data from dpd to ndpd numper of points per day

mean_pv =  pd.read_excel(path_input_data+file_name).mean().iloc[0]

In [9]:
PV_ts

array([nan, nan, nan, ..., nan, nan, nan])

### Plot Time Series

In [ ]:
# Create a Plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(y=PV_ts, mode='lines', name='PV',marker=dict(color=colors_dict['PV'])))
fig.add_trace(go.Scatter(y=Load_ts, mode='lines', name='Demand',marker=dict(color=colors_dict['Consumption'])))
fig.add_trace(go.Scatter(y=Wind_ts, mode='lines', name='Wind',marker=dict(color=colors_dict['Wind'])))
fig.update_layout(title=f'{country_name} 2021/2019', xaxis_title='Day', yaxis_title='Energy normalized')

# Show the plot
fig.show()

### Description of the problem

#### Equations:

- **Objective function** : 
  - Minimize dispatchable energy: $ \min(\sum{P_{dispatchable}(t)*dt}) $


- **Node Law** : 
  - $(P_{pv}(t) + P_{wind}(t) + P_{dispatchable}(t) - P_{in\_stock}(t) + P_{out\_stock}(t) = P_{demand}(t) + P_{curt}(t))$


- **State of charge**
  - $SOC(t+1)=SOC(t)+P_{in\_stock}(t) - P_{out\_stock}(t)$

#### Contraintes :
- $E_{wind} + E_{pv} \leq E_{demand}$
- We want a maximum storage size of 10 hours:  $E_{stock} \leq 10$
- Charging and discharging at the same time is not possible. 


### Implementation
#### Decision Variables:
- `x_pv`: Installed capacity for photovoltaic production.
- `x_wind`: Installed capacity for wind production.
- `ts_dispatchable`: Dispatchable production (can be controlled), time serie.
- `p_ch`: Battery charging power, time serie.
- `p_dech`: Battery discharging power, time serie.
- `SOC_ts`: State of charge of the battery, time serie.
- `p_curt`: Curtailment power (lost energy), time serie.
- `dech_active`: Binary variable indicating if the battery is charging or discharging.

### Run the optimization with GUROBI

**If the optimization has already been run, go to the next part where results can be loaded and analysed in plots.**

In [14]:
prob = LpProblem(f"myProblem_{country_name}", LpMinimize)
signal_length = len(Load_ts)

dt = 1 # hour
e_factor = 10 # max hours of consumption to be stored
E_max = e_factor*Load_ts.mean()# max capacity of storage (MW)
stock_efficiency = 0.8 # %
P_max = 100000 # MW used for the binary variable, voluntraily very high
capacity_max = 2*Load_ts.max()
cost_curtailment = 1e-3
# Decision Variables
x_pv = LpVariable("PV_coefficient", lowBound=0, upBound = capacity_max)
x_wind = LpVariable("Wind_coefficient", lowBound=0, upBound = capacity_max)
ts_dispatchable = LpVariable.dicts('Dispatchable_production', range(signal_length), lowBound=0, upBound = capacity_max)
p_ch = LpVariable.dicts('Pch', range(signal_length), lowBound=0, upBound = P_max)
p_dech = LpVariable.dicts('Pdech', range(signal_length), lowBound=0, upBound = P_max)
SOC_ts = LpVariable.dicts('Stock',range(signal_length), lowBound=0, upBound=E_max )
#TODO : changer upBound curtailment
p_curt = LpVariable.dicts('Curtailment',range(signal_length), lowBound=0)
dech_active = LpVariable.dicts('Dech_active', range(signal_length), cat='Binary')
max_dispatchable = LpVariable("Max Power Dispatchable", lowBound =0)

# Constraint 1: nodal law
for t in range(len(Load_ts)):
    prob += x_pv * PV_ts[t] + ts_dispatchable[t] + x_wind * Wind_ts[t]+p_dech[t] == Load_ts[t] +p_ch[t]+p_curt[t]

# Constraint 2: storage
for t in range(1, signal_length):
    prob += SOC_ts[t] == SOC_ts[t-1] + (stock_efficiency*p_ch[t]-p_dech[t])*dt
 
    # Binary variable: can't charge and discharge at the same time
    prob += p_ch[t] <= (1-dech_active[t])*P_max 
    prob += p_dech[t] <= (dech_active[t])*P_max

for t in range(signal_length):
    prob+= ts_dispatchable[t]<=max_dispatchable



#TODO : trouver une meilleure contrainte pour p_stock
prob+= p_ch[0]==0
prob+=p_dech[0]==0
prob += SOC_ts[0] == SOC_ts[signal_length-1] #same state of charge at the start and end

prob += x_pv*PV_ts.sum()+x_wind*Wind_ts.sum() == Load_ts.sum()

# Fonction objectif
prob += max_dispatchable+ cost_curtailment*lpSum(p_curt[t] for t in range(signal_length))

# Write problem in file 
prob.writeLP(f"myProblem_{country_name}.lp")


prob.solve(GUROBI(gapRel = 0.01))

# Afficher les résultats
print("Status:", LpStatus[prob.status])
print("Coefficient optimal pour PV:", x_pv.varValue)
print("Coefficient optimal pour Wind:", x_wind.varValue)

### Save the results

In [11]:
folder_path = f'results/{country_name}'
import os
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [12]:
# Get results of optimization 
optimized_pv = [x_pv.varValue * PV_ts[t] for t in range(signal_length)]
optimized_wind = [x_wind.varValue * Wind_ts[t] for t in range(signal_length)]
optimized_dispatchable = [ts_dispatchable[t].varValue for t in range(signal_length)]
optimized_stock = [SOC_ts[t].varValue for t in range(signal_length)]
optimized_p_curt = [p_curt[t].varValue for t in range(signal_length)]
optimized_charge = [p_ch[t].varValue for t in range(signal_length)]
optimized_discharge = [p_dech[t].varValue for t in range(signal_length)]

In [13]:
# Calculate energy totals
E_wind = np.sum(optimized_wind)*mean_load
E_pv = np.sum(optimized_pv)*mean_load
E_dispatch = np.sum(optimized_dispatchable)*mean_load
E_curt = np.sum(optimized_p_curt)*mean_load
E_loss  = (np.sum(optimized_charge)-np.sum(optimized_discharge))*mean_load
E_stock = np.sum(optimized_charge)*mean_load
E_destock = np.sum(optimized_discharge)*mean_load

In [15]:
E_dispatch

70413384.40275948

In [16]:
results = {
    'optimized_pv': optimized_pv,
    'optimized_wind': optimized_wind,
    'optimized_dispatchable': optimized_dispatchable,
    'optimized_stock':optimized_stock, 
    'optimized_charge': optimized_charge, 
    'optimized_discharge':optimized_discharge, 
    'optimized_p_curt' : optimized_p_curt,
    'consumption':np.array(Load_ts), 
    'pv_capacity': x_pv.varValue, 
    'wind_capacity': x_wind.varValue,
    'E_wind' : E_wind, 
    'E_pv':E_pv, 
    'E_dispatch':E_dispatch, 
    'E_curt':E_curt, 
    'E_loss':E_loss, 
    'E_stock':E_stock,
    'E_destock':E_destock
}

# !!!!!!!!!!!!!!!! CHANGE FILENAME HERE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
filename = f'results/{country_name}/optimization_results_optim_capacity_curt_cost_0305.pickle'

with open(filename, 'wb') as pickle_file:
    pickle.dump(results, pickle_file)

print(f"Optimization results saved to "+filename)

Optimization results saved to results/France/optimization_results_optim_capacity_curt_cost_0305.pickle


### Read results stored in pickle file

One can also use the dedicated notebook read_optimization_results.ipynb to do the following.

In [9]:
country_name = 'France'
data = pd.read_pickle(f'results/{country_name}/optimization_results_optim_capacity_0305.pickle')

In [10]:
optimized_pv = data['optimized_pv']
optimized_wind = data['optimized_wind']
optimized_charge = data['optimized_charge']
optimized_discharge = data['optimized_discharge']
optimized_p_curt = data['optimized_p_curt']
optimized_stock = data['optimized_stock']
optimized_dispatchable = data['optimized_dispatchable']
Load_ts = data['consumption']

E_wind = data['E_wind']/1000000
E_pv = data['E_pv']/1000000
E_dispatch = data['E_dispatch']/1000000
E_curt = data['E_curt']/1000000
E_loss  = data['E_loss']/1000000
E_stock = data['E_stock']/1000000
E_destock = data['E_destock']/1000000

## Plots

In [11]:
from plots import plot_ts_optim, plot_pie_energy, plot_storage, plot_stack_production

In [13]:
plot_ts_optim([optimized_pv, optimized_wind, optimized_dispatchable, optimized_p_curt,np.array(Load_ts) ], ['PV', 'Wind', 'Dispatchable', 'Curtailment', 'Consumption'], country_name, colors_dict = colors_dict,savefig=False)

In [14]:
plot_storage(optimized_charge, optimized_discharge, optimized_stock, country_name, colors_dict = colors_dict, savefig=False)

In [15]:
plot_stack_production(optimized_pv, optimized_wind, optimized_dispatchable, country_name,colors_dict = colors_dict, savefig = False)

## Diagrammes et graphes

In [16]:
# Bar chart in TWh
E_wind = data['E_wind']/1000000
E_pv = data['E_pv']/1000000
E_dispatch = data['E_dispatch']/1000000
E_curt = data['E_curt']/1000000
E_loss  = data['E_loss']/1000000
E_stock = data['E_stock']/1000000
E_destock = data['E_destock']/1000000

In [17]:
np.max(optimized_pv)*mean_load

102065.08986236191

In [18]:
np.mean(optimized_pv)/np.max(optimized_pv)

0.22006457097590662

In [19]:
plot_pie_energy([E_wind, E_pv, E_dispatch], country_name, colors_dict =colors_dict, savefig=False)

In [20]:

fig = go.Figure(data=[
    go.Bar(name='Renewables', x=[country_name], y=[E_wind+E_pv]),
    go.Bar(name='From stock', x=[country_name], y=[E_destock],marker=dict(color=colors_dict['SOC'])),
    go.Bar(name='Dispatchable', x=[country_name], y=[E_dispatch], marker=dict(color=colors_dict['Dispatchable'])),
    go.Bar(name='Curtailment', x=[country_name], y=[-E_curt], marker=dict(color=colors_dict['Curtailment'])),
    go.Bar(name='Loss', x=[country_name], y=[-E_loss])
])

fig.update_layout(
    title='Energy production',
    barmode='relative',
    yaxis_title = 'TWh'
)

fig.show()

In [27]:
# Sankey

sources = ['Wind', 'PV', 'Dispatchable','Production', 'Consumption', 'Loss', 'Curtailment']
E_production = E_wind + E_pv + E_dispatch
E_consumption = Load_ts.mean()

# Définition des valeurs d'énergie pour chaque source
values = [E_wind, E_pv, E_dispatch, E_production, E_consumption, E_loss, E_curt]  # Assurez-vous que la somme est égale à 100 ou à la valeur totale d'énergie

# Création des liens entre les sources
link_source = [0, 1, 2, 3, 3,  3]  # Les index 0, 1 et 2 représentent les sources Wind, PV et Dispatchable, respectivement.
link_target = [3, 3, 3, 4, 5, 6, 7 ]  # L'index 3 représente la source "Output"
link_value = values  # Les valeurs sont celles des énergies calculées
colors = ['rgba(31, 119, 180, 0.8)', 'rgba(255, 127, 14, 0.8)', 'rgba(44, 160, 44, 0.8)', 'rgba(214, 39, 40, 0.8)', 
          'rgba(148, 103, 189, 0.8)', 'rgba(140, 86, 75, 0.8)', 'rgba(227, 119, 194, 0.8)', 'rgba(127, 127, 127, 0.8)','rgba(44, 160, 44, 0.8)' ]

# Création du Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
      pad=15,
      thickness=20,
      line=dict(color="black", width=0.5),
      label=sources,
      color=colors
    ),
    link=dict(
      source=link_source,
      target=link_target,
      value=link_value
  ))])

# Mise en forme du layout
fig.update_layout(title_text="Energy Distribution Sankey Diagram")

fig.write_html(f"figures/{country_name}_optim_sankey_max.html")
# Afficher le diagramme
fig.show()